# BCI_002_Matriz_Comercial_B1_Factoring

## Información del Notebook

### Encabezado
**************************************************************************
* Nombre: 
* Ruta: 
* Autor: Esteban Gándara
* Ing. Rafael Montecinos - rafael.montecinost@bci.cl
* Fecha: 19/05/2025
* Descripción: 
* Documentación: 
***************************************************************************

### Mantenciones
**************************************************************************
#### Mantención Nro: 
* Autor: <Nombre Autor> (<Empresa del Autor (Bci/Otra)>) - Ing. SW BCI: <Nombre Ing. SW BCI>
* Fecha: <dd/mm/yyyy> 
* Descripción: <Descripción de la mantención>      
************************************************

### Tablas Entrada y Salida
**************************************************************************
#### Tablas Entrada: 
* 
* 
***************************************************************************
#### Tablas Salida: 
* 

## Captura de Variables

In [0]:
dbutils.widgets.removeAll()
dbutils.widgets.text("fechaProcesoW","","01 Fecha Proceso :")
dbutils.widgets.text("platinum_temp_dbW","","02 platinum temp db:")
dbutils.widgets.text("db_location_platinum_tempW","","03 Location DB Platinum Temp:")

### Asignar Objeto a Lectura de Widgets y Variables

In [0]:
fechaProcesoX = dbutils.widgets.get("fechaProcesoW")
spark.conf.set("bci.fechaProcesoX", fechaProcesoX)

platinum_temp_dbX = dbutils.widgets.get("platinum_temp_dbW")
spark.conf.set("bci.platinum_temp_dbX", platinum_temp_dbX)

db_location_platinum_tempX = dbutils.widgets.get("db_location_platinum_tempW")
spark.conf.set("bci.db_location_platinum_tempX", db_location_platinum_tempX)

print("*****Parámetros*****")
print("fechaProcesoX: " + fechaProcesoX)
print("platinum_temp_dbX: " + platinum_temp_dbX)
print("db_location_platinum_tempX: " + db_location_platinum_tempX)

*****Parámetros*****
fechaProcesoX: 20250530
platinum_temp_dbX: dsr_plt_bcitemp_db
db_location_platinum_tempX: abfss://platinum@bcirg2dlssbx.dfs.core.windows.net/plt_normativo_db/prv_hip/


### Librerías

In [0]:
import json
from datetime import datetime, date, time, timedelta
from dateutil.relativedelta import relativedelta

# Obtener fecha actual
current_date_str = datetime.now().strftime("%Y-%m-%d")
current_date = datetime.strptime(current_date_str, "%Y-%m-%d")
formatted_date = str(current_date.strftime("%Y%m%d"))

###Asigan Variables de fecha

In [0]:
ano = str(fechaProcesoX)[:4]
mes = str(fechaProcesoX)[4:][:2]
dia = str(fechaProcesoX)[6:][:2]
fechanormativo = str(ano+'-'+mes+'-'+dia)
fechacinta = str(dia+'-'+mes+'-'+ano)
anomes = str(ano+mes)
anomesdia = str(ano+mes+dia)
anomesdia2 = str(ano+'/'+mes+'/'+dia)


print("fecha_Formato1: " + ano)
print("fecha_Formato2: " + mes)
print("fecha_Formato3: " + dia)
print("fecha_Formato4: " + fechanormativo)
print("fecha_Formato5: " + fechacinta)
print("fecha_Formato6: " + anomes)
print("fecha_Formato7: " + anomesdia)
print("fecha_formato8: " + anomesdia2)

fecha_Formato1: 2025
fecha_Formato2: 05
fecha_Formato3: 30
fecha_Formato4: 2025-05-30
fecha_Formato5: 30-05-2025
fecha_Formato6: 202505
fecha_Formato7: 20250530
fecha_formato8: 2025/05/30


## Funciones

In [0]:
%run "./Funciones"

Fecha vacía

Fecha futura

## Validaciones

### Fecha vacía

In [0]:
fecha_vacia(fechaProcesoX)

### Fecha futura

In [0]:
fecha_futura(fechaProcesoX)

## Inicio de Lógica

### Creación de tablas temporales que se eliminarán al final de la ejecución

**************************************************************************

In [0]:
drop_tmp_002_CardetIrr = """ DROP TABLE IF EXISTS """ + platinum_temp_dbX + """.Tmp_CardetIrr """

In [0]:
sql_safe(drop_tmp_002_CardetIrr)

sql_safe: query ->  DROP TABLE IF EXISTS dsr_plt_bcitemp_db.Tmp_CardetIrr 


DataFrame[]

In [0]:
dbutils.fs.rm(db_location_platinum_tempX+"Tmp_CardetIrr", True)

True

In [0]:
create_tmp_002_Tmp_CardetIrr = """ CREATE TABLE IF NOT EXISTS """ + platinum_temp_dbX + """.Tmp_CardetIrr 
USING DELTA
PARTITIONED BY (dpf_fec_proceso)
LOCATION '""" + db_location_platinum_tempX + """Tmp_CardetIrr'
AS (
select 
dpf_fec_proceso
,cli_idc
,cli_vrt
,cli_rzn_soc
,ddr_idc
,ddr_vrt
,ddr_rzn_soc
,pdt_des_cra
,doc_num_documento
,doc_num_cuota
,Monto_Saldo_Cierre AS Deuda_TotalIFRS
,dod_cod_cobranza
,cod_aec
,dpf_ind_responsabilidad
,dpf_fec_vencimiento
,Clasificacion
,dod_ind_cartera
,IFNULL(dpf_des_cal_cliente,"") AS Clasificacion_Cliente
,IFNULL(dpf_des_cal_deudor,"") AS Clasificacion_Deudor
,Dias_Mora
,doc_num_operacion
,doc_id_documento
,doc_ind_fin_mes
,dpf_pct_pi
,dpf_pct_pdi
,dpf_mto_pe
,cli_rut
,Responsabilidad_CedenteFactoring
,Tipo_Prestamo
,Periodo_Id
,Ind_Castigo
,CASE WHEN trim(dod_ind_cartera)="INCUMPLIMIENTO" then 1 
else 0 END AS Colo_EsCardetIrr
, trim(cli_idc) || trim(cli_vrt) || trim(ddr_idc) || trim(ddr_vrt) || trim(pdt_des_cra) 
|| cast(doc_num_documento as string) || cast(doc_num_cuota as string) || cast(doc_num_operacion as string) AS Cod_niid
, CASE WHEN trim(Clasificacion_Deudor) ="A1" or trim(Clasificacion_Deudor) ="A2" or trim(Clasificacion_Deudor) ="A3" THEN 1
ELSE 0 END AS Calif_Deudor_Permitida
from """ + platinum_temp_dbX + """.Tmp_Factoring_Nodo
where Ind_Castigo=0 AND trim(Clasificacion)="G"
)
"""

In [0]:
sql_safe(create_tmp_002_Tmp_CardetIrr)

sql_safe: query ->  CREATE TABLE IF NOT EXISTS dsr_plt_bcitemp_db.Tmp_CardetIrr 
USING DELTA
PARTITIONED BY (dpf_fec_proceso)
LOCATION 'abfss://platinum@bcirg2dlssbx.dfs.core.windows.net/plt_normativo_db/prv_hip/Tmp_CardetIrr'
AS (
select 
dpf_fec_proceso
,cli_idc
,cli_vrt
,cli_rzn_soc
,ddr_idc
,ddr_vrt
,ddr_rzn_soc
,pdt_des_cra
,doc_num_documento
,doc_num_cuota
,Monto_Saldo_Cierre AS Deuda_TotalIFRS
,dod_cod_cobranza
,cod_aec
,dpf_ind_responsabilidad
,dpf_fec_vencimiento
,Clasificacion
,dod_ind_cartera
,IFNULL(dpf_des_cal_cliente,"") AS Clasificacion_Cliente
,IFNULL(dpf_des_cal_deudor,"") AS Clasificacion_Deudor
,Dias_Mora
,doc_num_operacion
,doc_id_documento
,doc_ind_fin_mes
,dpf_pct_pi
,dpf_pct_pdi
,dpf_mto_pe
,cli_rut
,Responsabilidad_CedenteFactoring
,Tipo_Prestamo
,Periodo_Id
,Ind_Castigo
,CASE WHEN trim(dod_ind_cartera)="INCUMPLIMIENTO" then 1 
else 0 END AS Colo_EsCardetIrr
, trim(cli_idc) || trim(cli_vrt) || trim(ddr_idc) || trim(ddr_vrt) || trim(pdt_des_cra) 
|| cast(doc_num_

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

## Mensaje Final

In [0]:
dbutils.notebook.exit("{\"coderror\":\"0\", \"msgerror\":\"Notebook termina ejecucion satisfactoriamente\"}")